In [47]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [48]:
FeatureSetB = pd.read_csv('FeatureSet/FS_48Hours.csv')

In [49]:
#Convert To hourly
from datetime import datetime, timedelta

now = datetime.now()

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))


In [50]:
FeatureSetB['date'] = pd.to_datetime(FeatureSetB.date, format='%Y-%m-%d')
FeatureSetB['time'] = pd.to_datetime(FeatureSetB.time, format='%H:%M:%S')

In [51]:
FeatureSetB['time'] = FeatureSetB['time'].apply(hour_rounder)
FeatureSetB['time'] = pd.to_datetime(FeatureSetB['time']).dt.time

In [52]:
FeatureSetB.drop(['date', 'time'], axis=1,inplace=True)

In [53]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetB.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)

In [54]:
result = []
for X in Subject_id:
    Feature = pd.DataFrame()
    Y = FeatureSetB.loc[FeatureSetB['subject_id'] == X]
    Z = Y.shape
    Rows = Z[0]
    result.append((X, Rows))
    

In [55]:
def aggregationFunc(tbl):
    Subject_id = tbl['subject_id'].mode()
    GCSEyesAgg = tbl['GCSEyes'].mode()
    GCSMotorAgg = tbl['GCSMotor'].mode()
    GCSVerbalAgg = tbl['GCSVerbal'].mode()
    Bilrubin_LevelAgg = tbl['Bilrubin_Level'].mean()
    BTAgg = tbl['BT'].mean()
    FIO2Agg = tbl['FIO2'].mean()
    HRAgg = tbl['HR'].mean()
    PAO2Agg = tbl['PAO2'].mean()
    Potassium_LevelAgg = tbl['Potassium_Level'].mean()
    SBC_LevelAgg = tbl['SBC_Level'].mean()                 
    SBPAgg = tbl['SBP'].mean()                 
    Sodium_LevelAgg = tbl['Sodium_Level'].mean()           
    SUN_LevelAgg = tbl['SUN_Level'].mean()               
    UrinAgg = tbl['Urin'].mean()   
    WBCC_LevelAgg = tbl['WBCC_Level'].mean()         
    Service_typeAgg = tbl['Service_type'].mode()               
    AgeAgg = tbl['Age'].mode()                       
    MetaStaticAgg = tbl['MetaStatic'].mode()                  
    HematologicAgg = tbl['Hematologic'].mode()                   
    Procedure_typeAgg = tbl['Procedure_type'].mode()                  
    X = pd.DataFrame({
    'subject_id': Subject_id,
    'Procedure_type': Procedure_typeAgg,
    'Bilrubin_Level': Bilrubin_LevelAgg,
    'BT': BTAgg,  
    'FIO2': FIO2Agg,
    'GCSEyes' : GCSEyesAgg,
    'GCSMotor' :  GCSMotorAgg,
    'GCSVerbal' : GCSVerbalAgg,
    'HR' : HRAgg,
    'PAO2': PAO2Agg, 
    'Potassium_Level': Potassium_LevelAgg,
    'SBC_Level': SBC_LevelAgg,
    'SBP': SBPAgg,
    'Service_type': Service_typeAgg,
    'Sodium_Level': Sodium_LevelAgg,         
    'SUN_Level': SUN_LevelAgg,              
    'Urin': UrinAgg,                      
    'WBCC_Level': WBCC_LevelAgg, 
    'Age': AgeAgg,
    'Hematologic' : HematologicAgg,
    'MetaStatic' : MetaStaticAgg,          
    })
    if(len(X.index) != 1):
        X.drop(1, inplace=True)
    return X

In [56]:
FeatureSetB['MetaStatic'] = FeatureSetB['MetaStatic'].fillna(0)

In [57]:
FeatureSetB['Hematologic'] = FeatureSetB['Hematologic'].fillna(0)

In [58]:
result_df = pd.DataFrame(result,columns =['subject_id','Rows'])

In [59]:
results_G48 = result_df.drop(result_df.index[result_df['Rows'] <= 48])
results_L48 = result_df.drop(result_df.index[result_df['Rows'] >= 48])
results_E48 = result_df.drop(result_df.index[result_df['Rows'] != 48])
results_G48.reset_index(drop=True,inplace=True)
results_L48.reset_index(drop=True,inplace=True)
results_E48.reset_index(drop=True,inplace=True)   

In [60]:
X1 = FeatureSetB.subject_id.isin(results_G48.subject_id)
G48 = FeatureSetB[X1]

In [61]:
from math import floor
import progressbar 
widgets=[' [', progressbar.Timer(), '] ',progressbar.Percentage(),progressbar.Bar(),' (', progressbar.ETA(), ') ',]
i = 0
FeatureSet = pd.DataFrame()
for S1 in progressbar.progressbar(results_G48.subject_id, widgets=widgets):
    #print('Subject_Id',S1)
    Y = G48.loc[G48['subject_id'] == S1]
    Y.reset_index(drop=True,inplace=True)
    #Check = Y.iloc[3]
    Rows = results_G48.Rows[i]
    i = i+1
    net = Rows - 48
    net2 = Rows - 2*48
    remove = list()
    remove2 = list()
    X1 =  pd.DataFrame()
    X2 =  pd.DataFrame()
    if(net2 <= 0):
        S2 = 0
        for SX in range(net):
            #print('S2 = ',S2)
            X1 = Y.iloc[[S2]]
            X2 = Y.iloc[[S2+1]]
            remove.append(S2+1)
            frames = [X1, X2]
            X3 = pd.concat(frames)
            X4 = aggregationFunc(X3)
            Y.iloc[S2] = X4
            Y.iloc[S2+1] = X4
            S2 = S2+2   
            if(S2>=(Rows-1)):
                break 
        Y.drop(remove, inplace=True)
        Y.reset_index(drop=True,inplace=True)         
    else:
        S4 = 0
        S3 = 0 
        Loop = floor(Rows/2)
        for SX in range(Loop):
            #print('S4 = ',S4)
            X1 = Y.iloc[S4]
            X2 = Y.iloc[S4+1]
            remove.append(S4+1)
            frames = [X1, X2]
            X3 = pd.concat(frames)
            X4 = aggregationFunc(X3)
            Y.iloc[S4] = X4
            Y.iloc[S4+1] = X4
            #Y.drop(S4+1, inplace=True) 
            S4 = S4+2
        Y.drop(remove, inplace=True)
        Y.reset_index(drop=True,inplace=True)
        NRows = Y.shape
        NRows = NRows[0]
        net3 = NRows - 48
        for SX in range(net3):
            #print('S3 = ',S3)
            X1 = Y.iloc[S3]
            X2 = Y.iloc[S3+1]
            remove2.append(S3+1)
            frames = [X1, X2]
            X3 = pd.concat(frames)
            X4 = aggregationFunc(X3)
            Y.iloc[S3] = X4
            Y.iloc[S3+1] = X4 
            S3 = S3+2
            if(S3>=(NRows-1)):
                break 
        Y.drop(remove2, inplace=True)
        Y.reset_index(drop=True,inplace=True) 
    FeatureSet = pd.concat([FeatureSet,Y])


 [Elapsed Time: 0:03:37] 100%|##############################| (Time:  0:03:37) 


In [62]:
X2 = FeatureSetB.subject_id.isin(results_L48.subject_id)
L48 = FeatureSetB[X2]

In [63]:
widgets=[' [', progressbar.Timer(), '] ',progressbar.Percentage(),progressbar.Bar(),' (', progressbar.ETA(), ') ',]
i = 0
FeatureSetL48 = pd.DataFrame()
for S1 in progressbar.progressbar(results_L48.subject_id, widgets=widgets):
    #print('Subject_Id',S1)
    Y = L48.loc[L48['subject_id'] == S1]
    Y.reset_index(drop=True,inplace=True)
    #Check = Y.iloc[3]
    Rows = results_L48.Rows[i]
    NeededRows = 48 - Rows
    new_index = pd.RangeIndex(NeededRows)
    Filler_df = pd.DataFrame(np.nan, index=new_index, columns=Y.columns)
    Filler_df.subject_id = S1
    frames = [Y, Filler_df]
    Y_New = pd.concat(frames)
    FeatureSetL48 = pd.concat([FeatureSetL48,Y_New])
    i = i+1
FeatureSetL48.reset_index(drop=True,inplace=True) 
    
    

 [Elapsed Time: 0:00:08] 100%|##############################| (Time:  0:00:08) 


In [64]:
# Add n blank rows
X3 = FeatureSetB.subject_id.isin(results_E48.subject_id)
E48 = FeatureSetB[X3]

In [65]:
frames = [FeatureSet, FeatureSetL48, E48]
FeatureSet48 = pd.concat(frames)

In [66]:
FeatureSetAGG = pd.DataFrame()
pd.options.mode.chained_assignment = None  # default='warn'
for S1 in Subject_id:
    Y = FeatureSet48.loc[FeatureSet48['subject_id'] == S1]
    X4 = aggregationFunc(Y)
    Y['subject_id'] = Y['subject_id'].fillna(X4.subject_id[0])
    Y['Bilrubin_Level'] = Y['Bilrubin_Level'].fillna(X4.Bilrubin_Level[0])
    Y['BT'] = Y['BT'].fillna(X4.BT[0])
    Y['subject_id'] = Y['subject_id'].fillna(X4.subject_id[0])
    Y['GCSEyes'] = Y['GCSEyes'].fillna(X4.GCSEyes[0])
    Y['GCSMotor'] = Y['GCSMotor'].fillna(X4.GCSMotor[0])
    Y['GCSVerbal'] = Y['GCSVerbal'].fillna(X4.GCSVerbal[0])
    Y['FIO2'] = Y['FIO2'].fillna(X4.FIO2[0])
    Y['HR'] = Y['HR'].fillna(X4.HR[0])
    Y['PAO2'] = Y['PAO2'].fillna(X4.PAO2[0])
    Y['Potassium_Level'] = Y['Potassium_Level'].fillna(X4.Potassium_Level[0])
    Y['SBC_Level'] = Y['SBC_Level'].fillna(X4.SBC_Level[0])      
    Y['SBP'] = Y['SBP'].fillna(X4.SBP[0])        
    Y['Sodium_Level'] = Y['Sodium_Level'].fillna(X4.Sodium_Level[0])        
    Y['SUN_Level'] = Y['SUN_Level'].fillna(X4.SUN_Level[0])       
    Y['Urin'] = Y['Urin'].fillna(X4.Urin[0])
    Y['WBCC_Level'] = Y['WBCC_Level'].fillna(X4.WBCC_Level[0])        
    Y['Service_type'] = Y['Service_type'].fillna(X4.Service_type[0])             
    Y['Age'] = Y['Age'].fillna(X4.Age[0])
    Y['MetaStatic'] = Y['MetaStatic'].fillna(X4.MetaStatic[0])              
    Y['Hematologic'] =  Y['Hematologic'].fillna(X4.Hematologic[0])               
    Y['Procedure_type'] = Y['Procedure_type'].fillna(X4.Procedure_type[0])
    FeatureSetAGG = pd.concat([FeatureSetAGG,Y])



In [67]:
FeatureSetAGG.to_csv('FeatureSet/AD_48Hours.csv', encoding='utf-8')